# Data cleaning process for COVID-19 in Mexico project

In [1]:
# Dependencies 
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib import cm
from  matplotlib.dates import DateFormatter, DayLocator, AutoDateLocator, AutoDateFormatter
from datetime import date, timedelta
from itertools import cycle
import sqlite3

In [2]:
# Reading data csv to pandas dataFrame
covid_file = "Resources/201122COVID19MEXICO.csv"
covid_df = pd.read_csv(covid_file, encoding = "latin-1")
covid_df

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,TOMA_MUESTRA,RESULTADO_LAB,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-11-22,1c4583,2,12,9,2,9,9,4,1,...,2,2,2,1,1,3,99,MÃ©xico,97,97
1,2020-11-22,0d55c9,2,12,9,1,9,9,16,1,...,2,2,2,1,1,3,99,MÃ©xico,97,97
2,2020-11-22,071735,2,9,21,2,21,21,114,1,...,2,2,2,1,1,3,99,MÃ©xico,97,97
3,2020-11-22,043f64,2,4,9,2,9,9,10,2,...,1,2,99,1,1,3,99,MÃ©xico,97,1
4,2020-11-22,0e07d8,1,4,15,2,15,15,104,2,...,2,2,99,1,1,3,99,MÃ©xico,97,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2701729,2020-11-22,3b6d0d,2,12,9,2,15,9,7,1,...,2,2,1,1,3,6,99,MÃ©xico,97,97
2701730,2020-11-22,2c0998,2,12,9,2,9,9,8,1,...,2,2,2,1,3,6,99,MÃ©xico,97,97
2701731,2020-11-22,2733e6,2,12,15,1,15,15,121,1,...,2,2,2,1,4,6,99,MÃ©xico,97,97
2701732,2020-11-22,399cda,1,4,30,1,30,30,39,1,...,2,2,2,2,97,6,99,MÃ©xico,97,97


In [3]:
for col in covid_df.columns: 
    print(col) 

FECHA_ACTUALIZACION
ID_REGISTRO
ORIGEN
SECTOR
ENTIDAD_UM
SEXO
ENTIDAD_NAC
ENTIDAD_RES
MUNICIPIO_RES
TIPO_PACIENTE
FECHA_INGRESO
FECHA_SINTOMAS
FECHA_DEF
INTUBADO
NEUMONIA
EDAD
NACIONALIDAD
EMBARAZO
HABLA_LENGUA_INDIG
INDIGENA
DIABETES
EPOC
ASMA
INMUSUPR
HIPERTENSION
OTRA_COM
CARDIOVASCULAR
OBESIDAD
RENAL_CRONICA
TABAQUISMO
OTRO_CASO
TOMA_MUESTRA
RESULTADO_LAB
CLASIFICACION_FINAL
MIGRANTE
PAIS_NACIONALIDAD
PAIS_ORIGEN
UCI


FECHA_ACTUALIZACION
ORIGEN
SECTOR
FECHA_SINTOMAS
NACIONALIDAD
EMBARAZO
HABLA_LENGUA_INDIG
INDIGENA
OTRAS_COM
OTRO_CASO
TOMA_MUESTRA
RESULTADO_LAB
MIGRANTE
PAIS_NACIONALIDAD
PAIS_ORIGEN
UCI

In [4]:
covid_df = covid_df.drop(['FECHA_ACTUALIZACION', 'ORIGEN', 'SECTOR', 'NACIONALIDAD', 'EMBARAZO', 
                          'HABLA_LENGUA_INDIG', 'INDIGENA', 'OTRA_COM', 'OTRO_CASO', 'TOMA_MUESTRA', 'RESULTADO_LAB',
                          'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'], axis = 1)
covid_df

,ID_REGISTRO,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,...,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,CLASIFICACION_FINAL
0,1c4583,9,2,9,9,4,1,2020-03-30,2020-03-30,9999-99-99,...,2,2,2,2,2,2,2,2,2,3
1,0d55c9,9,1,9,9,16,1,2020-03-26,2020-03-24,9999-99-99,...,2,2,2,2,2,2,2,2,2,3
2,071735,21,2,21,21,114,1,2020-03-18,2020-03-12,9999-99-99,...,1,2,2,2,2,1,2,2,2,3
3,043f64,9,2,9,9,10,2,2020-03-26,2020-03-26,2020-03-30,...,1,1,2,2,1,2,2,1,2,3
4,0e07d8,15,2,15,15,104,2,2020-03-28,2020-03-28,2020-04-02,...,2,2,2,2,2,2,2,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2701729,3b6d0d,9,2,15,9,7,1,2020-11-18,2020-11-08,9999-99-99,...,2,2,2,2,2,2,2,2,2,6
2701730,2c0998,9,2,9,9,8,1,2020-11-18,2020-11-18,9999-99-99,...,2,2,2,2,2,2,2,2,2,6
2701731,2733e6,15,1,15,15,121,1,2020-10-19,2020-10-17,9999-99-99,...,2,2,2,2,2,2,2,2,2,6
2701732,399cda,30,1,30,30,39,1,2020-10-19,2020-10-16,9999-99-99,...,2,2,2,2,2,2,2,2,2,6


In [5]:
covid_df.dtypes

ID_REGISTRO            object
ENTIDAD_UM              int64
SEXO                    int64
ENTIDAD_NAC             int64
ENTIDAD_RES             int64
MUNICIPIO_RES           int64
TIPO_PACIENTE           int64
FECHA_INGRESO          object
FECHA_SINTOMAS         object
FECHA_DEF              object
INTUBADO                int64
NEUMONIA                int64
EDAD                    int64
DIABETES                int64
EPOC                    int64
ASMA                    int64
INMUSUPR                int64
HIPERTENSION            int64
CARDIOVASCULAR          int64
OBESIDAD                int64
RENAL_CRONICA           int64
TABAQUISMO              int64
CLASIFICACION_FINAL     int64
dtype: object

In [6]:
covid_df["FECHA_SINTOMAS"] = pd.to_datetime(covid_df['FECHA_SINTOMAS'])
covid_df = covid_df.replace(['9999-99-99'],'NA')
covid_df.dtypes

ID_REGISTRO                    object
ENTIDAD_UM                      int64
SEXO                            int64
ENTIDAD_NAC                     int64
ENTIDAD_RES                     int64
MUNICIPIO_RES                   int64
TIPO_PACIENTE                   int64
FECHA_INGRESO                  object
FECHA_SINTOMAS         datetime64[ns]
FECHA_DEF                      object
INTUBADO                        int64
NEUMONIA                        int64
EDAD                            int64
DIABETES                        int64
EPOC                            int64
ASMA                            int64
INMUSUPR                        int64
HIPERTENSION                    int64
CARDIOVASCULAR                  int64
OBESIDAD                        int64
RENAL_CRONICA                   int64
TABAQUISMO                      int64
CLASIFICACION_FINAL             int64
dtype: object

In [7]:
covid_df.head()

,ID_REGISTRO,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,...,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,CLASIFICACION_FINAL
0,1c4583,9,2,9,9,4,1,2020-03-30,2020-03-30,NA,...,2,2,2,2,2,2,2,2,2,3
1,0d55c9,9,1,9,9,16,1,2020-03-26,2020-03-24,NA,...,2,2,2,2,2,2,2,2,2,3
2,071735,21,2,21,21,114,1,2020-03-18,2020-03-12,NA,...,1,2,2,2,2,1,2,2,2,3
3,043f64,9,2,9,9,10,2,2020-03-26,2020-03-26,2020-03-30,...,1,1,2,2,1,2,2,1,2,3
4,0e07d8,15,2,15,15,104,2,2020-03-28,2020-03-28,2020-04-02,...,2,2,2,2,2,2,2,2,2,3


In [8]:
covid_df = covid_df.sort_values(["FECHA_SINTOMAS"])
covid_df

,ID_REGISTRO,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,...,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,CLASIFICACION_FINAL
66439,01e884,8,1,8,8,19,1,2020-01-03,2020-01-01,NA,...,2,2,2,2,2,2,2,2,2,6
6240,05c7cb,9,2,25,9,14,2,2020-01-03,2020-01-01,NA,...,2,2,2,2,2,2,2,2,1,7
31193,03706c,14,1,14,14,98,2,2020-01-01,2020-01-01,2020-01-10,...,2,2,2,2,2,2,2,2,2,7
31407,1dc68d,14,1,14,14,101,2,2020-01-07,2020-01-01,NA,...,2,2,2,2,2,2,2,2,2,7
54897,15d446,22,1,9,22,17,1,2020-01-06,2020-01-01,NA,...,2,2,2,2,2,2,2,2,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2610841,2ca34c,9,2,9,9,7,1,2020-11-22,2020-11-22,NA,...,2,2,2,2,2,2,2,2,1,6
2593888,3b2580,27,1,27,27,4,1,2020-11-22,2020-11-22,NA,...,2,2,2,2,2,2,2,2,2,6
2585694,3b0517,9,2,9,15,39,1,2020-11-22,2020-11-22,NA,...,2,2,2,2,2,2,2,2,1,6
2674073,29d96c,27,1,27,27,4,1,2020-11-22,2020-11-22,NA,...,2,2,1,2,2,2,1,2,1,6


In [9]:
dates = pd.date_range(date(2020,1,1),date(2020,11,23) - timedelta(days=1), freq='d')

In [10]:
covid_clean_df = pd.DataFrame()
covid_clean_df['Dates'] = dates
covid_clean_df

,Dates
0,2020-01-01
1,2020-01-02
2,2020-01-03
3,2020-01-04
4,2020-01-05
...,...
322,2020-11-18
323,2020-11-19
324,2020-11-20
325,2020-11-21


In [11]:
covid_clean_df = covid_clean_df.loc[covid_clean_df.index.repeat(36)]
covid_clean_df

,Dates
0,2020-01-01
0,2020-01-01
0,2020-01-01
0,2020-01-01
0,2020-01-01
...,...
326,2020-11-22
326,2020-11-22
326,2020-11-22
326,2020-11-22


In [12]:
covid_clean_df = covid_clean_df.reset_index()
del covid_clean_df['index']
num_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
            28, 29, 30, 31, 32, 36, 97, 98, 99]
State = []
index = covid_clean_df.index
number_of_rows = len(index)

for i in range(int(number_of_rows/len(num_list))):
    for num in num_list:
        State.append(num)
covid_clean_df['State'] = State
actives_df = covid_clean_df.copy()
recovered_df = covid_clean_df.copy()
age_ranges_df = covid_clean_df.copy()
covid_clean_df

,Dates,State
0,2020-01-01,1
1,2020-01-01,2
2,2020-01-01,3
3,2020-01-01,4
4,2020-01-01,5
...,...,...
11767,2020-11-22,32
11768,2020-11-22,36
11769,2020-11-22,97
11770,2020-11-22,98


In [13]:
len(State)

11772

In [14]:
number_of_rows/len(num_list)

327.0

### Confirmed Count

In [15]:
confirmed_df = covid_df.copy()
confirmed_index_names = confirmed_df[(covid_df['CLASIFICACION_FINAL'] != 1) & 
                                 (covid_df['CLASIFICACION_FINAL'] != 2) &
                                 (covid_df['CLASIFICACION_FINAL'] != 3)].index 
confirmed_df.drop(confirmed_index_names, inplace = True) 
female_age_count_df = confirmed_df.copy()
male_age_count_df = confirmed_df.copy()
confirmed_df

,ID_REGISTRO,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,...,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,CLASIFICACION_FINAL
250,17235f,19,2,19,19,46,1,2020-01-13,2020-01-13,NA,...,2,2,2,2,2,2,2,2,2,3
2496,0f634f,25,1,25,25,6,1,2020-01-29,2020-01-29,NA,...,2,2,2,2,2,2,2,2,2,3
48592,18efbb,9,2,22,22,14,2,2020-02-10,2020-02-06,2020-05-12,...,1,2,2,2,1,2,2,2,2,1
48690,032db1,21,2,21,21,114,1,2020-02-22,2020-02-18,NA,...,2,2,2,2,2,2,1,2,2,1
428,189846,15,2,9,15,104,1,2020-02-28,2020-02-19,NA,...,2,2,2,2,2,2,2,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056063,2b01ef,27,2,27,27,4,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,2,2,1,2,2,3
2112292,31c1fd,31,1,31,31,50,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,2,2,2,2,2,3
2233585,3d79fa,19,1,19,19,9,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,1,2,1,2,2,3
2125346,37395c,21,1,21,21,114,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,2,2,2,2,2,3


In [16]:
confirmed_grouped_df = confirmed_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Confirmed').reset_index()
confirmed_grouped_df = confirmed_grouped_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})
#confirmed_grouped_df

In [17]:
covid_clean_df = pd.merge(covid_clean_df, confirmed_grouped_df, on=['Dates', 'State'], how='left')
covid_clean_df["Confirmed"] = covid_clean_df["Confirmed"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed
0,2020-01-01,1,0.0
1,2020-01-01,2,0.0
2,2020-01-01,3,0.0
3,2020-01-01,4,0.0
4,2020-01-01,5,0.0


### Negative Count

In [18]:
negative_df = covid_df.copy()
negative_index_names = negative_df[covid_df['CLASIFICACION_FINAL'] != 7].index 
negative_df.drop(negative_index_names, inplace = True) 
#negative_df

In [19]:
negative_df = negative_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Negatives').reset_index()
negative_df = negative_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})
#negative_df

In [20]:
covid_clean_df = pd.merge(covid_clean_df, negative_df, on=['Dates', 'State'], how='left')
covid_clean_df["Negatives"] = covid_clean_df["Negatives"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives
0,2020-01-01,1,0.0,3.0
1,2020-01-01,2,0.0,2.0
2,2020-01-01,3,0.0,3.0
3,2020-01-01,4,0.0,0.0
4,2020-01-01,5,0.0,5.0


### Suspicious Count

In [21]:
suspicious_df = covid_df.copy()
suspicious_index_names = suspicious_df[(covid_df['CLASIFICACION_FINAL'] != 4) & 
                                       (covid_df['CLASIFICACION_FINAL'] != 5) &
                                       (covid_df['CLASIFICACION_FINAL'] != 6)].index 
suspicious_df.drop(suspicious_index_names, inplace = True) 
#suspicious_df.head()

In [22]:
suspicious_df = suspicious_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Suspicious').reset_index()
suspicious_df = suspicious_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})
#suspicious_df

In [23]:
covid_clean_df = pd.merge(covid_clean_df, suspicious_df, on=['Dates', 'State'], how='left')
covid_clean_df["Suspicious"] = covid_clean_df["Suspicious"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious
0,2020-01-01,1,0.0,3.0,2.0
1,2020-01-01,2,0.0,2.0,2.0
2,2020-01-01,3,0.0,3.0,1.0
3,2020-01-01,4,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0


### Death Count (Confirmed Cases Only)

In [24]:
confirmed_death_df = confirmed_df.copy()
death_index_names = confirmed_death_df[(confirmed_df['FECHA_DEF'] == 'NA')].index 
confirmed_death_df.drop(death_index_names, inplace = True) 

In [25]:
death_df = confirmed_death_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Deaths').reset_index()
death_df = death_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})
#death_df

In [26]:
covid_clean_df = pd.merge(covid_clean_df, death_df, on=['Dates', 'State'], how='left')
covid_clean_df["Deaths"] = covid_clean_df["Deaths"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths
0,2020-01-01,1,0.0,3.0,2.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0


### Male Confirmed Cases

In [27]:
male_df = confirmed_df.copy()
male_index_names = male_df[(confirmed_df['SEXO'] != 2)].index 
male_df.drop(male_index_names, inplace = True) 
#male_df.head()

In [28]:
male_df = male_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Males').reset_index()
male_df = male_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})
#male_df

In [29]:
covid_clean_df = pd.merge(covid_clean_df, male_df, on=['Dates', 'State'], how='left')
covid_clean_df["Males"] = covid_clean_df["Males"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0


### Female Confirmed Cases

In [30]:
female_df = confirmed_df.copy()
female_index_names = female_df[(confirmed_df['SEXO'] != 1)].index 
female_df.drop(female_index_names, inplace = True) 
#female_df.head()

In [31]:
female_df = female_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Females').reset_index()
female_df = female_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})
#female_df

In [32]:
covid_clean_df = pd.merge(covid_clean_df, female_df, on=['Dates', 'State'], how='left')
covid_clean_df["Females"] = covid_clean_df["Females"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0


### Hospitalized Confirmed Cases

In [33]:
hospitalized_df = confirmed_df.copy()
hospitalized_index_names = hospitalized_df[(confirmed_df['TIPO_PACIENTE'] != 2)].index 
hospitalized_df.drop(hospitalized_index_names, inplace = True) 
#hospitalized_df.head()

In [34]:
hospitalized_df = hospitalized_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Hospitalized').reset_index()
hospitalized_df = hospitalized_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})
#hospitalized_df

In [35]:
covid_clean_df = pd.merge(covid_clean_df, hospitalized_df, on=['Dates', 'State'], how='left')
covid_clean_df["Hospitalized"] = covid_clean_df["Hospitalized"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0


### Confirmed Outpatients

In [36]:
outpatients_df = confirmed_df.copy()
outpatients_index_names = outpatients_df[(confirmed_df['TIPO_PACIENTE'] != 1)].index 
outpatients_df.drop(outpatients_index_names, inplace = True) 
outpatients_df = outpatients_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Outpatients').reset_index()
outpatients_df = outpatients_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [37]:
covid_clean_df = pd.merge(covid_clean_df, outpatients_df, on=['Dates', 'State'], how='left')
covid_clean_df["Outpatients"] = covid_clean_df["Outpatients"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0


### Confirmed Patients with Pneumonia

In [38]:
pneumonia_df = confirmed_df.copy()
pneumonia_index_names = pneumonia_df[(confirmed_df['NEUMONIA'] != 1)].index 
pneumonia_df.drop(pneumonia_index_names, inplace = True) 
pneumonia_df = pneumonia_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Pneumonia').reset_index()
pneumonia_df = pneumonia_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [39]:
covid_clean_df = pd.merge(covid_clean_df, pneumonia_df, on=['Dates', 'State'], how='left')
covid_clean_df["Pneumonia"] = covid_clean_df["Pneumonia"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Confirmed Patients with Diabetes

In [40]:
diabetes_df = confirmed_df.copy()
diabetes_index_names = diabetes_df[(confirmed_df['DIABETES'] != 1)].index 
diabetes_df.drop(diabetes_index_names, inplace = True) 
diabetes_df = diabetes_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Diabetes').reset_index()
diabetes_df = diabetes_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [41]:
covid_clean_df = pd.merge(covid_clean_df, diabetes_df, on=['Dates', 'State'], how='left')
covid_clean_df["Diabetes"] = covid_clean_df["Diabetes"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia,Diabetes
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Confirmed Patients with Chronic Obstructive Pulmonary Disease (COPD)

In [42]:
copd_df = confirmed_df.copy()
copd_index_names = copd_df[(confirmed_df['EPOC'] != 1)].index 
copd_df.drop(copd_index_names, inplace = True) 
copd_df = copd_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('COPD').reset_index()
copd_df = copd_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [43]:
covid_clean_df = pd.merge(covid_clean_df, copd_df, on=['Dates', 'State'], how='left')
covid_clean_df["COPD"] = covid_clean_df["COPD"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia,Diabetes,COPD
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Confirmed Patients with Asthma

In [44]:
asthma_df = confirmed_df.copy()
asthma_index_names = asthma_df[(confirmed_df['ASMA'] != 1)].index 
asthma_df.drop(asthma_index_names, inplace = True) 
asthma_df = asthma_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Asthma').reset_index()
asthma_df = asthma_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [45]:
covid_clean_df = pd.merge(covid_clean_df, asthma_df, on=['Dates', 'State'], how='left')
covid_clean_df["Asthma"] = covid_clean_df["Asthma"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia,Diabetes,COPD,Asthma
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Confirmed Patients with Immunosuppression

In [46]:
immusuppr_df = confirmed_df.copy()
immusuppr_index_names = immusuppr_df[(confirmed_df['INMUSUPR'] != 1)].index 
immusuppr_df.drop(immusuppr_index_names, inplace = True) 
immusuppr_df = immusuppr_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Immunosuppression').reset_index()
immusuppr_df = immusuppr_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [47]:
covid_clean_df = pd.merge(covid_clean_df, immusuppr_df, on=['Dates', 'State'], how='left')
covid_clean_df["Immunosuppression"] = covid_clean_df["Immunosuppression"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia,Diabetes,COPD,Asthma,Immunosuppression
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Confirmed Patients with Hypertension

In [48]:
hypertension_df = confirmed_df.copy()
hypertension_index_names = hypertension_df[(confirmed_df['HIPERTENSION'] != 1)].index 
hypertension_df.drop(hypertension_index_names, inplace = True) 
hypertension_df = hypertension_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Hypertension').reset_index()
hypertension_df = hypertension_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [49]:
covid_clean_df = pd.merge(covid_clean_df, hypertension_df, on=['Dates', 'State'], how='left')
covid_clean_df["Hypertension"] = covid_clean_df["Hypertension"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia,Diabetes,COPD,Asthma,Immunosuppression,Hypertension
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Confirmed Patients with a Cardiovascular Disease

In [50]:
cardiovascular_df = confirmed_df.copy()
cardiovascular_index_names = cardiovascular_df[(confirmed_df['CARDIOVASCULAR'] != 1)].index 
cardiovascular_df.drop(cardiovascular_index_names, inplace = True) 
cardiovascular_df = cardiovascular_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Cardiovascular').reset_index()
cardiovascular_df = cardiovascular_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [51]:
covid_clean_df = pd.merge(covid_clean_df, cardiovascular_df, on=['Dates', 'State'], how='left')
covid_clean_df["Cardiovascular"] = covid_clean_df["Cardiovascular"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia,Diabetes,COPD,Asthma,Immunosuppression,Hypertension,Cardiovascular
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Confirmed Patients with Obesity

In [52]:
obesity_df = confirmed_df.copy()
obesity_index_names = obesity_df[(confirmed_df['OBESIDAD'] != 1)].index 
obesity_df.drop(obesity_index_names, inplace = True) 
obesity_df = obesity_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Obesity').reset_index()
obesity_df = obesity_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [53]:
covid_clean_df = pd.merge(covid_clean_df, obesity_df, on=['Dates', 'State'], how='left')
covid_clean_df["Obesity"] = covid_clean_df["Obesity"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia,Diabetes,COPD,Asthma,Immunosuppression,Hypertension,Cardiovascular,Obesity
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Confirmed Patients with Chronic Kidney Disease (CKD)

In [54]:
ckd_df = confirmed_df.copy()
ckd_index_names = ckd_df[(confirmed_df['RENAL_CRONICA'] != 1)].index 
ckd_df.drop(ckd_index_names, inplace = True) 
ckd_df = ckd_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('CKD').reset_index()
ckd_df = ckd_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [55]:
covid_clean_df = pd.merge(covid_clean_df, ckd_df, on=['Dates', 'State'], how='left')
covid_clean_df["CKD"] = covid_clean_df["CKD"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia,Diabetes,COPD,Asthma,Immunosuppression,Hypertension,Cardiovascular,Obesity,CKD
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Confirmed Smoker Patients

In [56]:
smokers_df = confirmed_df.copy()
smokers_index_names = smokers_df[(confirmed_df['TABAQUISMO'] != 1)].index 
smokers_df.drop(smokers_index_names, inplace = True) 
smokers_df = smokers_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Smokers').reset_index()
smokers_df = smokers_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [57]:
covid_clean_df = pd.merge(covid_clean_df, smokers_df, on=['Dates', 'State'], how='left')
covid_clean_df["Smokers"] = covid_clean_df["Smokers"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia,Diabetes,COPD,Asthma,Immunosuppression,Hypertension,Cardiovascular,Obesity,CKD,Smokers
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Confirmed Intubated Patients

In [58]:
intubated_df = confirmed_df.copy()
intubated_index_names = intubated_df[(confirmed_df['INTUBADO'] != 1)].index 
intubated_df.drop(intubated_index_names, inplace = True) 
intubated_df = intubated_df.groupby(['ENTIDAD_RES','FECHA_SINTOMAS']).size().to_frame('Intubated').reset_index()
intubated_df = intubated_df.rename(columns={"FECHA_SINTOMAS": "Dates", "ENTIDAD_RES": "State"})

In [59]:
covid_clean_df = pd.merge(covid_clean_df, intubated_df, on=['Dates', 'State'], how='left')
covid_clean_df["Intubated"] = covid_clean_df["Intubated"].fillna(0)
covid_clean_df.head()

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,...,Diabetes,COPD,Asthma,Immunosuppression,Hypertension,Cardiovascular,Obesity,CKD,Smokers,Intubated
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Active Confirmed Cases (Up to that Day)

In [60]:
actives_df.head()

,Dates,State
0,2020-01-01,1
1,2020-01-01,2
2,2020-01-01,3
3,2020-01-01,4
4,2020-01-01,5


In [61]:
actives_confirmed_df = confirmed_df.copy()
actives_confirmed_df

,ID_REGISTRO,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,...,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,CLASIFICACION_FINAL
250,17235f,19,2,19,19,46,1,2020-01-13,2020-01-13,NA,...,2,2,2,2,2,2,2,2,2,3
2496,0f634f,25,1,25,25,6,1,2020-01-29,2020-01-29,NA,...,2,2,2,2,2,2,2,2,2,3
48592,18efbb,9,2,22,22,14,2,2020-02-10,2020-02-06,2020-05-12,...,1,2,2,2,1,2,2,2,2,1
48690,032db1,21,2,21,21,114,1,2020-02-22,2020-02-18,NA,...,2,2,2,2,2,2,1,2,2,1
428,189846,15,2,9,15,104,1,2020-02-28,2020-02-19,NA,...,2,2,2,2,2,2,2,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056063,2b01ef,27,2,27,27,4,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,2,2,1,2,2,3
2112292,31c1fd,31,1,31,31,50,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,2,2,2,2,2,3
2233585,3d79fa,19,1,19,19,9,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,1,2,1,2,2,3
2125346,37395c,21,1,21,21,114,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,2,2,2,2,2,3


### Recovered Confirmed Cases (Up to that Day)

In [62]:
recovered_df.head()

,Dates,State
0,2020-01-01,1
1,2020-01-01,2
2,2020-01-01,3
3,2020-01-01,4
4,2020-01-01,5


In [63]:
recovered_confirmed_df = covid_df.copy()
recovered_index_names = recovered_confirmed_df[(covid_df['CLASIFICACION_FINAL'] != 1) & 
                                               (covid_df['CLASIFICACION_FINAL'] != 2) & 
                                               (covid_df['CLASIFICACION_FINAL'] != 3)].index 
recovered_confirmed_df.drop(recovered_index_names, inplace = True) 
recovered_index_names = recovered_confirmed_df[(covid_df['TIPO_PACIENTE'] != 1)].index 
recovered_confirmed_df.drop(recovered_index_names, inplace = True) 
recovered_index_names = recovered_confirmed_df[(covid_df['FECHA_DEF'] != 'NA')].index 
recovered_confirmed_df.drop(recovered_index_names, inplace = True) 
recovered_confirmed_df

/Users/alfredo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/alfredo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,ID_REGISTRO,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,...,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,CLASIFICACION_FINAL
250,17235f,19,2,19,19,46,1,2020-01-13,2020-01-13,NA,...,2,2,2,2,2,2,2,2,2,3
2496,0f634f,25,1,25,25,6,1,2020-01-29,2020-01-29,NA,...,2,2,2,2,2,2,2,2,2,3
48690,032db1,21,2,21,21,114,1,2020-02-22,2020-02-18,NA,...,2,2,2,2,2,2,1,2,2,1
428,189846,15,2,9,15,104,1,2020-02-28,2020-02-19,NA,...,2,2,2,2,2,2,2,2,2,3
2316,044969,25,1,25,25,6,1,2020-02-26,2020-02-21,NA,...,2,2,2,2,2,2,2,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056063,2b01ef,27,2,27,27,4,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,2,2,1,2,2,3
2112292,31c1fd,31,1,31,31,50,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,2,2,2,2,2,3
2233585,3d79fa,19,1,19,19,9,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,1,2,1,2,2,3
2125346,37395c,21,1,21,21,114,1,2020-11-20,2020-11-20,NA,...,2,2,2,2,2,2,2,2,2,3


### Actives 2

In [64]:
num_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
            28, 29, 30, 31, 32]
actives_df = pd.DataFrame(num_list, columns=['State'])
recovered_df = actives_df.copy()
age_ranges_2 = actives_df.copy()
actives_df

,State
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [65]:
for index, row in actives_df.iterrows():
    temp_actives = actives_confirmed_df[(actives_confirmed_df['FECHA_SINTOMAS'] > np.datetime64('2020-11-22') - np.timedelta64(14,'D')) &
                                        (actives_confirmed_df['ENTIDAD_RES'] == row['State'])]
    aux_index = temp_actives.index
    actives_number = len(aux_index)
    actives_df.at[index, 'Actives'] = actives_number
actives_df


,State,Actives
0,1,549.0
1,2,423.0
2,3,442.0
3,4,47.0
4,5,1298.0
5,6,79.0
6,7,42.0
7,8,487.0
8,9,3957.0
9,10,1343.0


### Recovered 2

In [66]:
for index, row in recovered_df.iterrows():
    temp_recovered = recovered_confirmed_df[(recovered_confirmed_df['FECHA_SINTOMAS'] <= np.datetime64('2020-11-22') - np.timedelta64(14,'D')) &
                                          (recovered_confirmed_df['ENTIDAD_RES'] == row['State'])]
    aux_index = temp_recovered.index
    recovered_number = len(aux_index)
    recovered_df.at[index, 'Recovered'] = recovered_number
recovered_df


,State,Recovered
0,1,8526.0
1,2,16806.0
2,3,11935.0
3,4,4628.0
4,5,31503.0
5,6,5176.0
6,7,5003.0
7,8,20306.0
8,9,152734.0
9,10,15103.0


### Age & Gender Distribution

In [67]:
female_index = female_age_count_df[(covid_df['SEXO'] != 1)].index 
female_age_count_df.drop(female_index, inplace = True)
female_age_count_df.reset_index()

/Users/alfredo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,index,ID_REGISTRO,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,...,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,CLASIFICACION_FINAL
0,2496,0f634f,25,1,25,25,6,1,2020-01-29,2020-01-29,...,2,2,2,2,2,2,2,2,2,3
1,2316,044969,25,1,25,25,6,1,2020-02-26,2020-02-21,...,2,2,2,2,2,2,2,2,2,3
2,1067,0900ba,25,1,25,25,6,1,2020-03-24,2020-02-24,...,2,2,2,2,2,2,2,2,2,3
3,1992,12a243,9,1,9,9,12,1,2020-02-24,2020-02-24,...,2,2,2,2,2,2,2,2,2,3
4,1890,0aad01,7,1,9,7,101,1,2020-02-29,2020-02-25,...,2,2,2,2,2,2,2,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510085,2044079,3a3957,9,1,9,9,17,1,2020-11-20,2020-11-20,...,2,2,2,2,2,2,2,2,2,3
510086,2105252,306426,19,1,19,19,39,2,2020-11-20,2020-11-20,...,2,2,2,2,2,2,2,2,2,3
510087,2112292,31c1fd,31,1,31,31,50,1,2020-11-20,2020-11-20,...,2,2,2,2,2,2,2,2,2,3
510088,2233585,3d79fa,19,1,19,19,9,1,2020-11-20,2020-11-20,...,2,2,2,2,1,2,1,2,2,3


In [68]:
male_index = male_age_count_df[(covid_df['SEXO'] != 2)].index 
male_age_count_df.drop(male_index, inplace = True)
male_age_count_df.reset_index()

/Users/alfredo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,index,ID_REGISTRO,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,...,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,CLASIFICACION_FINAL
0,250,17235f,19,2,19,19,46,1,2020-01-13,2020-01-13,...,2,2,2,2,2,2,2,2,2,3
1,48592,18efbb,9,2,22,22,14,2,2020-02-10,2020-02-06,...,1,2,2,2,1,2,2,2,2,1
2,48690,032db1,21,2,21,21,114,1,2020-02-22,2020-02-18,...,2,2,2,2,2,2,1,2,2,1
3,428,189846,15,2,9,15,104,1,2020-02-28,2020-02-19,...,2,2,2,2,2,2,2,2,2,3
4,238,05f3d3,9,2,9,9,7,2,2020-02-27,2020-02-22,...,2,2,2,2,2,2,2,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531780,2109944,1f9ad2,9,2,9,9,10,1,2020-11-20,2020-11-20,...,2,2,2,2,2,2,2,2,2,3
531781,2174928,3e2dc4,32,2,32,32,17,2,2020-11-20,2020-11-20,...,1,2,2,2,1,1,1,2,2,3
531782,2213967,381452,31,2,31,31,50,1,2020-11-20,2020-11-20,...,2,2,2,2,2,2,2,2,2,3
531783,2056063,2b01ef,27,2,27,27,4,1,2020-11-20,2020-11-20,...,2,2,2,2,2,2,1,2,2,3


In [69]:
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, np.inf]
labels=['0-5', '6-10' , '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', 
        '56-60', '61-65', '66-70', '71-75', '76-80', '81-85','More than 85']
b = pd.cut(female_age_count_df['EDAD'], bins=bins, labels=labels, include_lowest=True)
female_df = female_age_count_df.groupby(['ENTIDAD_RES', b]).size().unstack(fill_value=0).stack().reset_index(name='Female Count')
b2 = pd.cut(male_age_count_df['EDAD'], bins=bins, labels=labels, include_lowest=True)
male_df = male_age_count_df.groupby(['ENTIDAD_RES', b2]).size().unstack(fill_value=0).stack().reset_index(name='Male Count')
age_ranges_df = pd.merge(female_df, male_df, on=['ENTIDAD_RES', 'EDAD'], how='left')
age_ranges_df.reset_index(inplace=True)
age_ranges_df = age_ranges_df.rename(columns={"ENTIDAD_RES": "State_ID", "EDAD": "Age Range", "index": "PK"})
age_ranges_df


,PK,State_ID,Age Range,Female Count,Male Count
0,0,1,0-5,59,52
1,1,1,6-10,27,41
2,2,1,11-15,52,46
3,3,1,16-20,159,155
4,4,1,21-25,581,539
...,...,...,...,...,...
571,571,32,66-70,308,319
572,572,32,71-75,214,244
573,573,32,76-80,150,207
574,574,32,81-85,103,108


In [70]:
covid_clean_df

,Dates,State,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,...,Diabetes,COPD,Asthma,Immunosuppression,Hypertension,Cardiovascular,Obesity,CKD,Smokers,Intubated
0,2020-01-01,1,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,2,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01,3,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01,5,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11767,2020-11-22,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11768,2020-11-22,36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11769,2020-11-22,97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11770,2020-11-22,98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
sql_df = covid_clean_df.copy()
sql_df = sql_df.drop(['Dates'], axis = 1)
sql_df = sql_df.groupby(['State']).sum()
sql_df.reset_index(inplace=True)
sql_df = sql_df.drop([32, 33, 34, 35])

sql_df = pd.merge(sql_df, actives_df, on=['State'], how='left')
sql_df["Actives"] = sql_df["Actives"].fillna(0)
sql_df = pd.merge(sql_df, recovered_df, on=['State'], how='left')
sql_df["Recovered"] = sql_df["Recovered"].fillna(0)

sql_df = sql_df.rename(columns={"State": "State_ID"})
sql_df = sql_df.append(sql_df.sum(numeric_only=True), ignore_index=True)
sql_df.xs(32)['State_ID'] = 33
sql_df

,State_ID,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia,...,Asthma,Immunosuppression,Hypertension,Cardiovascular,Obesity,CKD,Smokers,Intubated,Actives,Recovered
0,1.0,11933.0,22638.0,7739.0,1006.0,5927.0,6006.0,2793.0,9140.0,1594.0,...,334.0,133.0,2474.0,226.0,1822.0,362.0,1056.0,454.0,549.0,8526.0
1,2.0,25086.0,17370.0,25100.0,4172.0,12393.0,12693.0,7893.0,17193.0,7034.0,...,1017.0,300.0,7309.0,659.0,5426.0,520.0,1892.0,1575.0,423.0,16806.0
2,3.0,13846.0,23758.0,2844.0,653.0,6829.0,7017.0,1368.0,12478.0,1176.0,...,440.0,71.0,2464.0,183.0,2815.0,137.0,788.0,333.0,442.0,11935.0
3,4.0,6703.0,9604.0,3838.0,898.0,3925.0,2778.0,1971.0,4732.0,1479.0,...,245.0,95.0,1408.0,136.0,1403.0,99.0,314.0,271.0,47.0,4628.0
4,5.0,39098.0,48287.0,11557.0,3057.0,19099.0,19999.0,6191.0,32907.0,3274.0,...,688.0,277.0,5890.0,462.0,4364.0,522.0,2134.0,698.0,1298.0,31503.0
5,6.0,7017.0,5509.0,2000.0,738.0,3642.0,3375.0,1733.0,5284.0,1305.0,...,193.0,58.0,1436.0,119.0,1581.0,190.0,492.0,371.0,79.0,5176.0
6,7.0,7676.0,6809.0,27833.0,1172.0,4310.0,3366.0,2552.0,5124.0,1528.0,...,265.0,80.0,1766.0,169.0,1473.0,226.0,417.0,624.0,42.0,5003.0
7,8.0,27982.0,17021.0,8393.0,3159.0,14121.0,13861.0,7126.0,20856.0,5302.0,...,827.0,331.0,5992.0,653.0,4237.0,529.0,2117.0,1200.0,487.0,20306.0
8,9.0,187229.0,331887.0,88543.0,12988.0,93059.0,94170.0,29658.0,157571.0,24776.0,...,4214.0,1851.0,28795.0,3350.0,29095.0,2540.0,20794.0,6390.0,3957.0,152734.0
9,10.0,18821.0,22581.0,4098.0,1059.0,8764.0,10057.0,2323.0,16498.0,1089.0,...,470.0,101.0,2702.0,222.0,2540.0,205.0,1019.0,353.0,1343.0,15103.0


In [72]:
states_file = "Resources/states_info.csv"
states_df = pd.read_csv(states_file, encoding = "latin-1")
states_df

,State_ID,State_Name,State_Abb,Latitude,Longitude
0,1,AGUASCALIENTES,AS,22.000000,-102.500000
1,2,BAJA CALIFORNIA,BC,30.033892,-115.142511
2,3,BAJA CALIFORNIA SUR,BS,25.581801,-111.570616
3,4,CAMPECHE,CC,19.329206,-89.943915
4,5,COAHUILA DE ZARAGOZA,CL,27.333333,-102.000000
5,6,COLIMA,CM,19.166667,-104.000000
6,7,CHIAPAS,CS,16.500000,-92.500000
7,8,CHIHUAHUA,CH,28.500000,-106.000000
8,9,CIUDAD DE MEXICO,DF,19.432630,-99.133178
9,10,DURANGO,DG,24.833333,-104.833333


In [73]:
conn = sqlite3.connect('covid_db.sqlite')
sql_df.to_sql('covid_mexico', conn, if_exists='replace', index=False)
states_df.to_sql('states_info', conn, if_exists='replace', index=False)
age_ranges_df.to_sql('ages_distribution', conn, if_exists='replace', index=False)
pd.read_sql('select * from covid_mexico', conn)

/Users/alfredo/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


,State_ID,Confirmed,Negatives,Suspicious,Deaths,Males,Females,Hospitalized,Outpatients,Pneumonia,...,Asthma,Immunosuppression,Hypertension,Cardiovascular,Obesity,CKD,Smokers,Intubated,Actives,Recovered
0,1.0,11933.0,22638.0,7739.0,1006.0,5927.0,6006.0,2793.0,9140.0,1594.0,...,334.0,133.0,2474.0,226.0,1822.0,362.0,1056.0,454.0,549.0,8526.0
1,2.0,25086.0,17370.0,25100.0,4172.0,12393.0,12693.0,7893.0,17193.0,7034.0,...,1017.0,300.0,7309.0,659.0,5426.0,520.0,1892.0,1575.0,423.0,16806.0
2,3.0,13846.0,23758.0,2844.0,653.0,6829.0,7017.0,1368.0,12478.0,1176.0,...,440.0,71.0,2464.0,183.0,2815.0,137.0,788.0,333.0,442.0,11935.0
3,4.0,6703.0,9604.0,3838.0,898.0,3925.0,2778.0,1971.0,4732.0,1479.0,...,245.0,95.0,1408.0,136.0,1403.0,99.0,314.0,271.0,47.0,4628.0
4,5.0,39098.0,48287.0,11557.0,3057.0,19099.0,19999.0,6191.0,32907.0,3274.0,...,688.0,277.0,5890.0,462.0,4364.0,522.0,2134.0,698.0,1298.0,31503.0
5,6.0,7017.0,5509.0,2000.0,738.0,3642.0,3375.0,1733.0,5284.0,1305.0,...,193.0,58.0,1436.0,119.0,1581.0,190.0,492.0,371.0,79.0,5176.0
6,7.0,7676.0,6809.0,27833.0,1172.0,4310.0,3366.0,2552.0,5124.0,1528.0,...,265.0,80.0,1766.0,169.0,1473.0,226.0,417.0,624.0,42.0,5003.0
7,8.0,27982.0,17021.0,8393.0,3159.0,14121.0,13861.0,7126.0,20856.0,5302.0,...,827.0,331.0,5992.0,653.0,4237.0,529.0,2117.0,1200.0,487.0,20306.0
8,9.0,187229.0,331887.0,88543.0,12988.0,93059.0,94170.0,29658.0,157571.0,24776.0,...,4214.0,1851.0,28795.0,3350.0,29095.0,2540.0,20794.0,6390.0,3957.0,152734.0
9,10.0,18821.0,22581.0,4098.0,1059.0,8764.0,10057.0,2323.0,16498.0,1089.0,...,470.0,101.0,2702.0,222.0,2540.0,205.0,1019.0,353.0,1343.0,15103.0


In [74]:
pd.read_sql('select * from states_info', conn)

,State_ID,State_Name,State_Abb,Latitude,Longitude
0,1,AGUASCALIENTES,AS,22.000000,-102.500000
1,2,BAJA CALIFORNIA,BC,30.033892,-115.142511
2,3,BAJA CALIFORNIA SUR,BS,25.581801,-111.570616
3,4,CAMPECHE,CC,19.329206,-89.943915
4,5,COAHUILA DE ZARAGOZA,CL,27.333333,-102.000000
5,6,COLIMA,CM,19.166667,-104.000000
6,7,CHIAPAS,CS,16.500000,-92.500000
7,8,CHIHUAHUA,CH,28.500000,-106.000000
8,9,CIUDAD DE MEXICO,DF,19.432630,-99.133178
9,10,DURANGO,DG,24.833333,-104.833333


In [75]:
pd.read_sql('select * from ages_distribution', conn)

,PK,State_ID,Age Range,Female Count,Male Count
0,0,1,0-5,59,52
1,1,1,6-10,27,41
2,2,1,11-15,52,46
3,3,1,16-20,159,155
4,4,1,21-25,581,539
...,...,...,...,...,...
571,571,32,66-70,308,319
572,572,32,71-75,214,244
573,573,32,76-80,150,207
574,574,32,81-85,103,108
